In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip3 install nlpaug
import tarfile
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint 
import numpy as np
import datetime
import re
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip3 install nlpaug
import tarfile

In [3]:
!wget https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt

--2021-11-08 09:10:13--  https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ddkmtqz01jc024u/glove.6B.100d.txt [following]
--2021-11-08 09:10:13--  https://www.dropbox.com/s/raw/ddkmtqz01jc024u/glove.6B.100d.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3725d4c711009ad5a98b27fff3.dl.dropboxusercontent.com/cd/0/inline/BZmQqumjcyGaMRWuwvKIQnzVK2hS1NwPc-7T-ZFq4hUvz0myA0hn3huQVEmCTuMhmVMydGIY9_ONqQrgiL87RRg2DS8lDS9d0kaxPAPbaKfaQyxZUvLEY2CamO7fBoX1jE7DaHqoY5keMebYXjfw9IHb/file# [following]
--2021-11-08 09:10:13--  https://uc3725d4c711009ad5a98b27fff3.dl.dropboxusercontent.com/cd/0/inline/BZmQqumjcyGaMRWuwvKIQnzVK2hS1NwPc-7T-ZFq4hUvz0myA0hn3huQVEmCTuMhmVMydGIY9_O

In [4]:
!curl --header "Host: www.cl.cam.ac.uk" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en;q=0.9" --header "Referer: https://www.cl.cam.ac.uk/research/nl/bea2019st/" "https://www.cl.cam.ac.uk/research/nl/bea2019st/data/fce_v2.1.bea19.tar.gz" -L -o "fce_v2.1.bea19.tar.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2709k  100 2709k    0     0  2347k      0  0:00:01  0:00:01 --:--:-- 2347k


In [5]:
import tarfile
fname='/content/fce_v2.1.bea19.tar.gz'
fname.endswith("tar.gz")
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [6]:
!curl --header "Host: www.cl.cam.ac.uk" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-GB,en;q=0.9" --header "Referer: https://www.cl.cam.ac.uk/research/nl/bea2019st/" "https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.1.bea19.tar.gz" -L -o "wi+locness_v2.1.bea19.tar.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5977k  100 5977k    0     0  4541k      0  0:00:01  0:00:01 --:--:-- 4541k


In [7]:
import tarfile
fname='/content/wi+locness_v2.1.bea19.tar.gz'
fname.endswith("tar.gz")
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [8]:
def m2_to_txt(path):
  '''
  this function processes the .m2 format file 
  and converts the required sentence into txt format'''
  m2 = open(path).read().strip().split("\n\n")
  corrupted = open('corrupted.txt', "w")
  uncorrupted=open('uncorrupted.txt','w')
  skip = {"noop", "UNK", "Um"}
  for sent in m2:
    sent=sent.split('\n')
  
    incorr_sent=sent[0].split()[1:]
  
    corrupted.write(" ".join(incorr_sent)+"\n")
    edits = sent[1:]
  
    offset=0
  
    for edit in edits:
		    edit = edit.split("|||")
		    if edit[1] in skip: continue # Ignore certain edits
			
			
		    span = edit[0].split()[1:] # Ignore "A "
		    start = int(span[0])
		    end = int(span[1])
		    cor = edit[2].split()
		    incorr_sent[start+offset:end+offset] = cor
      
		    offset = offset-(end-start)+len(cor)
      
    uncorrupted.write(" ".join(incorr_sent)+"\n")


In [9]:
#extracting data from m2 files
m2_to_txt('/content/fce/m2/fce.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data1=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/fce/m2/fce.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data2=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/N.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data3=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/C.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data4=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/B.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data5=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/ABCN.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data6=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/A.dev.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data7=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])

m2_to_txt('/content/wi+locness/m2/A.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data8=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])


m2_to_txt('/content/wi+locness/m2/B.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data9=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])


m2_to_txt('/content/wi+locness/m2/ABC.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data10=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])


m2_to_txt('/content/wi+locness/m2/C.train.gold.bea19.m2')
with open('uncorrupted.txt','r') as file:
  uncorrupted=[]
  line=file.readlines()
  for lines in line:
    uncorrupted.append(" ".join(lines.split()[:len(lines)]))
with open('corrupted.txt','r') as file:
  corrupted=[]
  line=file.readlines()
  for lines in line:
    corrupted.append(" ".join(lines.split()[:len(lines)]))

data11=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])




In [10]:
#compiling all the datasets into one
data=pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11],ignore_index=True)

In [11]:
#removing rows having same data in both columns
data=data[data['corrupted']!=data['uncorrupted']]

In [12]:
#https://stackoverflow.com/questions/30887979/i-want-to-create-a-script-for-unzip-tar-gz-file-via-python
import tarfile
fname='sm_norm_mt.tar.gz'
fname.endswith("tar.gz")
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()


In [13]:
#https://stackoverflow.com/questions/53987653/read-a-text-file-contents-from-second-line-in-python
with open('/content/release/en2cn-2k.en2nen2cn','r') as file:
  line=file.readlines()
corrupted=[]
uncorrupted=[]
for i,lines in enumerate(line):
  if i%3==0:
    corrupted.append(lines.strip())
  elif (i%3==1):
    uncorrupted.append(lines.strip())
  else:
    pass
  


In [14]:
data2=pd.DataFrame(zip(corrupted,uncorrupted),columns=['corrupted','uncorrupted'])
#data=pd.concat([data,data12],ignore_index=True)

In [15]:
data1= data.sample(frac=1).reset_index(drop=True)
data1=data1.drop_duplicates()

In [16]:
data1.to_pickle('data1.pkl')
data2.to_pickle('data2.pkl')

In [17]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()
        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        self.embedding = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,initial_state):
          '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state

          '''
          input_embedd                           = self.embedding(input_sequence)
          self.lstm_output, self.state_h,self.state_c = self.lstm(input_embedd)
          return self.lstm_output, self.state_h,self.state_c

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.state_h=tf.zeros([batch_size])
      self.state_c=tf.zeros([batch_size])
      return self.state_h,self.state_c
      

In [18]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):

    super().__init__()
    self.att_units=att_units
    self.scoring_function=scoring_function
    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      
      pass
    if scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.W=Dense(self.att_units)  
      
      pass
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.W1=Dense(self.att_units,activation='tanh')
      self.W2=Dense(self.att_units,activation='tanh')  
    
      self.V=Dense(1)
      pass
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        #print(encoder_output.shape,decoder_hidden_state.shape)
        decoder_hidden_state=tf.reshape(decoder_hidden_state,shape=(tf.shape(decoder_hidden_state)[0],1,tf.shape(decoder_hidden_state)[1]))
        #print(decoder_hidden_state.shape[0])
        #print(decoder_hidden_state.shape[1])
        #print(decoder_hidden_state.shape)
        score=tf.matmul(decoder_hidden_state,encoder_output,transpose_b=True)
        
        
    elif self.scoring_function == 'general':
        # Implement General score function here
        decoder_hidden_state=tf.reshape(decoder_hidden_state,shape=(tf.shape(decoder_hidden_state)[0],1,tf.shape(decoder_hidden_state)[1]))
        score=tf.matmul(decoder_hidden_state,self.W(encoder_output),transpose_b=True)
       

    elif self.scoring_function == 'concat':
        # Implement General score function here
        decoder=tf.expand_dims(decoder_hidden_state,1)
        score=self.V(tf.nn.tanh(self.W1(decoder)+self.W2(encoder_output)))
        attention_weights=tf.nn.softmax(score,axis=1)
        context_vector=attention_weights*encoder_output
        context_vector=tf.reduce_sum(context_vector,axis=1)

        return context_vector, attention_weights

        
    attention_weights=tf.nn.softmax(score,axis=2)
    context_vector=tf.matmul(attention_weights,encoder_output)
    context_vector=tf.reshape(context_vector,shape=(tf.shape(context_vector)[0],tf.shape(context_vector)[2]))
    attention_weights=tf.reshape(attention_weights,shape=(tf.shape(attention_weights)[0],tf.shape(attention_weights)[2],tf.shape(attention_weights)[1]))
    return context_vector, attention_weights
    
    

In [19]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      super().__init__()
      # Initialize decoder embedding layer, LSTM and any other objects needed
      self.embedding = Embedding(input_dim=tar_vocab_size, output_dim=embedding_dim, input_length=input_length,
                           mask_zero=True, name="embedding_layer")
      self.lstm=LSTM(dec_units,return_sequences=True,return_state=True)
      self.attention=Attention(score_fun,att_units)
      self.dense=tf.keras.layers.Dense(tar_vocab_size)


  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    x=self.embedding(input_to_decoder)
    context_vector,attention_weights=self.attention(state_h,encoder_output)
    concat=tf.concat([x,tf.expand_dims(context_vector,1)],axis=-1)
    output,decoder_state_h,decoder_state_c=self.lstm(x,initial_state=[state_h,state_c])
    output=tf.reshape(output,(-1,output.shape[2]))
    final_output=self.dense(output)
    return final_output,decoder_state_h,decoder_state_c,attention_weights,context_vector


In [20]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder,self).__init__()
      self.out_vocab_size=out_vocab_size
      self.embedding_dim=embedding_dim
      self.input_length=input_length
      self.dec_units=dec_units
      self.score_fun=score_fun
      self.att_units=att_units
      self.One_Step_Decoder=One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        all_outputs=tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1])
        for i in range(tf.shape(input_to_decoder)[1]):
          output,decoder_hidden_state,decoder_cell_state,attention_weights,context_vector=self.One_Step_Decoder(input_to_decoder[:,i:i+1],encoder_output,decoder_hidden_state,decoder_cell_state)
          
          all_outputs=all_outputs.write(i,output)
        all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs

In [21]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,inp_vocab_size, embedding_dim,enc_units,input_length,out_vocab_size,dec_units ,score_fun ,att_units,batch_size):
    #Intialize objects from encoder decoder
    super().__init__()
    self.batch_size=batch_size
    self.encoder=Encoder(inp_vocab_size,embedding_dim,enc_units,input_length)
    self.decoder=Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)

  
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    
    input,output=data[0],data[1]
    initial_state=self.encoder.initialize_states(self.batch_size)
    encoder_output,encoder_final_state_h,encoder_final_state_c=self.encoder(input,initial_state)
    decoder_outputs=self.decoder(output,encoder_output,encoder_final_state_h,encoder_final_state_c)

    return decoder_outputs

In [22]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [23]:
from tensorflow.keras import backend as K
def accuracy(y_true, y_pred):

    pred_value= K.cast(K.argmax(y_pred, axis=-1), dtype='float32')
    true_value = K.cast(K.equal(y_true, pred_value), dtype='float32')

    mask = K.cast(K.greater(y_true, 0), dtype='float32')
    n_correct = K.sum(mask * true_value)
    n_total = K.sum(mask)
  
    return n_correct / n_total

In [24]:
class Dataset:
    def __init__(self, data, tok_corrupt, tok_uncorrupt, max_len):
        self.encoder_inps = data['corrupted'].values
        self.decoder_inps = data['uncorrupted_inp'].values
        self.decoder_outs = data['uncorrupted_op'].values
        self.tok_corrupt = tok_corrupt
        self.tok_uncorrupt = tok_uncorrupt
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tok_corrupt.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tok_uncorrupt.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tok_uncorrupt.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))

    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [82]:
def model_inference(data1,data2):
  '''the function takes single argument and performs all the preprocessing and feature
  engineering on top of that and also performs modelling of data..
  the function will return the predicted output of our task'''

  #HERE THE FUNCTION ARGUMENT WILL TAKE A DATAFRAME AS AN INPUT

  #PERFORMING ALL THE REQUIRED DATA PREPROCESSING
  
  def decontractions(phrase):
    '''decontracted takes text and convert contractions into natural form.'''
  #https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490
  # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

  # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase
  data1['corrupted']=data1['corrupted'].apply(lambda x:decontractions(x))
  data1['uncorrupted']=data1['uncorrupted'].apply(lambda x:decontractions(x))

  data2['corrupted']=data2['corrupted'].apply(lambda x:decontractions(x))
  data2['uncorrupted']=data2['uncorrupted'].apply(lambda x:decontractions(x))

  data1['corrupted']=data1['corrupted'].apply(lambda x:x.lower())
  data1['uncorrupted']=data1['uncorrupted'].apply(lambda x:x.lower())

  data2['corrupted']=data2['corrupted'].apply(lambda x:x.lower())
  data2['uncorrupted']=data2['uncorrupted'].apply(lambda x:x.lower())

  data1['corrupted_len'] = data1['corrupted'].str.split().apply(len)
  data1 = data1[data1['corrupted_len'] < 25]

  data1['uncorrupted_len'] = data1['uncorrupted'].str.split().apply(len)
  data1 = data1[data1['uncorrupted_len'] < 25]

  data2['corrupted_len'] = data2['corrupted'].str.split().apply(len)
  data2 = data2[data2['corrupted_len'] < 25]

  data2['uncorrupted_len'] = data2['uncorrupted'].str.split().apply(len)
  data2 = data2[data2['uncorrupted_len'] < 25]

  data1['uncorrupted_inp']=data1['uncorrupted'].apply(lambda x:('<START>'+' '+x))
  data1['uncorrupted_op']=data1['uncorrupted'].apply(lambda x:(x+' '+'<END>'))
  data1.drop(['uncorrupted'],axis=1,inplace=True)

  data2['uncorrupted_inp']=data2['uncorrupted'].apply(lambda x:('<START>'+' '+x))
  data2['uncorrupted_op']=data2['uncorrupted'].apply(lambda x:(x+' '+'<END>'))
  data2.drop(['uncorrupted'],axis=1,inplace=True)

  data1.drop(['corrupted_len','uncorrupted_len'],axis=1,inplace=True)
  data2.drop(['corrupted_len','uncorrupted_len'],axis=1,inplace=True)


  data=data1.sample(5000)
  
  train, validation = train_test_split(data, test_size=0.2)
  train=pd.concat([train,data2.iloc[0:1500]],ignore_index=True).astype(str)
  validation=pd.concat([validation,data2.iloc[1500:1675]],ignore_index=True).astype(str)


  train.to_pickle('train.pkl')
  validation.to_pickle('validation.pkl')

  train=pd.read_pickle('train.pkl')
  validation=pd.read_pickle('validation.pkl')

  # for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
  # with this we can use only one tokenizer for both encoder output and decoder output
  train.iloc[0]['uncorrupted_inp']= str(train.iloc[0]['uncorrupted_inp'])+' <end>'
  train.iloc[0]['uncorrupted_out']= str(train.iloc[0]['uncorrupted_op'])+' <end>'

  train.sample(frac=1)

  from tensorflow.keras.preprocessing.text import Tokenizer
  tok_corrupted=Tokenizer(filters='#$%&()*+-/=@[\\]^_`{|}~\t\n')
  tok_corrupted.fit_on_texts(train['corrupted'].values)

  tok_uncorrupted=Tokenizer(filters='#$%&()*+-/=@[\\]^_`{|}~\t\n')
  tok_uncorrupted.fit_on_texts(train['uncorrupted_inp'].values)

  vocab_corrupt=len(tok_corrupted.word_index.keys())
  vocab_uncorrupt=len(tok_uncorrupted.word_index.keys())

  embeddings_index = dict()
  f = open('glove.6B.100d.txt')
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
  f.close()

  embedding_matrix = np.zeros((vocab_corrupt+1, 100))
  for word, i in tok_corrupted.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector


  train_dataset = Dataset(train, tok_corrupted, tok_uncorrupted,25 )
  test_dataset  = Dataset(validation,tok_corrupted, tok_uncorrupted,25)

  

  train_dataloader = Dataloder(train_dataset, batch_size=64)
  test_dataloader = Dataloder(test_dataset, batch_size=64)
  
  print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)


  import datetime
  from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping,TerminateOnNaN,TensorBoard
  earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
  filepath="/content/drive/MyDrive/CaseStudy2/Model5/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
  checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=4, save_best_only=True, mode='auto')
  logdir = "/content/drive/MyDrive/CaseStudy2/Model5/Logs/fit_model2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  train_summary_writer = tf.summary.create_file_writer(logdir)
  tensorboard_callback = TensorBoard(log_dir=logdir,histogram_freq=1,profile_batch = 100000000)
  callback_list = [checkpoint,tensorboard_callback]


  model  = encoder_decoder(vocab_corrupt,100,300,25,vocab_uncorrupt,300,'concat',50,64)
  optimizer = tf.keras.optimizers.Adam()
  model.compile(optimizer=optimizer,loss=loss_function,metrics=[accuracy])
  batch_size=64
  train_steps=train.shape[0]//batch_size
  valid_steps=validation.shape[0]//batch_size
  model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=1, validation_data=train_dataloader, validation_steps=valid_steps)
  model.summary()

  #model.save_weights('model.h5')
  model.load_weights('model.h5')

  #https://matplotlib.org/3.1.1/gallery/ticks_and_spines/tick-locators.html
  import matplotlib.pyplot as plt
  import matplotlib.ticker as ticker

  def plot_attention(attention, sentence, predicted_sentence):
    #Refer: https://www.tensorflow.org/tutorials/text/nmt_with_attention#translate
    
    #attention_mat= attention[:len(predicted_sentence), :len(sentence)]
    
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show() 

  def predict(input_sentence):
    input_vec = tok_corrupted.texts_to_sequences([input_sentence])
    input_vec =  tf.keras.preprocessing.sequence.pad_sequences(input_vec,maxlen=25,padding='post')
    input_vec = tf.convert_to_tensor(input_vec)
    enc_initial_states = None
    enc_out, h_state, c_state = model.layers[0](input_vec,enc_initial_states)
    curr_vec = np.array(tok_uncorrupted.word_index["<start>"])
    curr_vec = curr_vec.reshape(1,1)
    end_vec = np.array(tok_uncorrupted.word_index["<end>"]).reshape(1,1)
    result=''
    atten_weights= np.zeros((25,25)) #(max_target_length, max_source_length)
    for i in range(25):
      decoder_output, lstm_state_h, lstm_state_c, atten_w, context_vectors = model.layers[1].One_Step_Decoder(curr_vec, enc_out, h_state, c_state)
    
      Result_beam_list=beam_search_decoder(decoder_output,k=1)
      Result_beam=Result_beam_list[0][0]
   
      predicted_id = tf.argmax(decoder_output[0]).numpy()
      
    
      if tok_uncorrupted.index_word[predicted_id] != '<end>':
        result += tok_uncorrupted.index_word[Result_beam[0]] + ' '
      # the predicted ID is fed back into the model
      atten_w = tf.squeeze(atten_w, axis=-1) #to get shape as (batch, time_step)
        
      atten_w = tf.squeeze(atten_w, axis=0)
      
      atten_weights[i][:atten_w.shape[0]] = atten_w.numpy()
      
      h_state, c_state = lstm_state_h, lstm_state_c
      curr_vec = tf.expand_dims([predicted_id], 0)
    return result,input_sentence, atten_weights

  def translate_sentence(inp_sentence):
    result, input_sentence, attention_plot = predict(inp_sentence)
    print('Input: %s' % (input_sentence))
    print('Predicted translation: {}'.format(result))
    print("-"*50)
    attention_plot = attention_plot[:len(result.split(' ')), :len(input_sentence.split(' '))]
    plot_attention(attention_plot, input_sentence.split(' '),    result.split(' '))
    return result


  from tqdm import tqdm
  import nltk.translate.bleu_score as bleu
  test=validation.sample(10)
  score=[]
  predicted=[]
  test_inp=test['corrupted'].values

  for i in tqdm(test_inp):
  
    pred,inp,att=predict(i)
    predicted.append(pred)

  for i in list(zip(test['uncorrupted_inp'].values,predicted)):
    score.append(bleu.sentence_bleu([i[0].split(' ')[1:]],i[1].split(' ')))
  
  print("average bleu score:",(sum(score)/len(score)))

In [83]:
model_inference(data1,data2)

(64, 25) (64, 25) (64, 25)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


85/85 [==============================] - 52s 486ms/step - loss: 4.0644 - accuracy: 0.0831 - val_loss: 3.7798 - val_accuracy: 0.0914
Model: "encoder_decoder_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_16 (Encoder)         multiple                  1362800   
_________________________________________________________________
decoder_16 (Decoder)         multiple                  3476746   
Total params: 4,839,546
Trainable params: 4,839,546
Non-trainable params: 0
_________________________________________________________________


100%|██████████| 10/10 [16:16<00:00, 97.62s/it]

average bleu score: 0.5253437739541507



/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
